<a href="https://colab.research.google.com/github/Ayona123/Drug_repurposing/blob/main/trial_mol100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kora -q
import kora.install.rdkit

     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 


In [2]:
from rdkit import Chem

In [3]:
!pip install git+https://github.com/samoturk/mol2vec

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-8bh11yxn
  Running command git clone -q https://github.com/samoturk/mol2vec /tmp/pip-req-build-8bh11yxn
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14038 sha256=b18834c7cb18e8af75f29c37992e337ff767c450520af4b872e96bc8d88e964a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_sbq4b0t/wheels/80/d5/e1/90dc211e4a17780f99eaec43328470a70284cd731ef2a77034
Successfully built mol2vec


In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
import pandas as pd
df=pd.read_csv('/content/sample_data/mol100.csv')

target=df['Vina Score']
df2=df[['smiles']]
df2['mol'] = df2['smiles'].apply(lambda x: Chem.MolFromSmiles(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
from gensim.models import word2vec
from mol2vec import features
from mol2vec import helpers
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
df2['sentence'] = df2.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [8]:
model = word2vec.Word2Vec.load('/content/sample_data/model_300dim.pkl')
import numpy as np
df2['mol2vec'] = [DfVec(x) for x in sentences2vec(df2['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in df2['mol2vec']])
y = target.values
X.shape

(100, 100)

In [11]:
model = word2vec.Word2Vec.load('/content/sample_data/model_300dim.pkl')
import numpy as np
df2['mol2vec'] = [DfVec(x) for x in sentences2vec(df2['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in df2['mol2vec']])
y = target.values
X

array([[  2.388134  ,   1.3088104 ,  -5.648465  , ...,  20.603825  ,
        -18.554123  ,  -3.9436667 ],
       [ -5.9474406 ,  12.177692  ,  -2.9627283 , ...,  24.682535  ,
        -12.525246  ,  -7.3815465 ],
       [  0.23757131,   3.458234  ,   3.4650526 , ...,  10.698219  ,
         -5.5967464 ,   0.06362966],
       ...,
       [  2.2638395 ,  -1.0280678 ,  -5.6789074 , ...,  17.989702  ,
        -11.478041  ,  -3.682911  ],
       [ -0.8210803 ,   6.1485424 ,  -2.5667288 , ...,  20.460728  ,
        -12.7525625 ,  -1.8041596 ],
       [  1.7967522 ,   2.7830381 ,  -4.825134  , ...,  13.8512125 ,
         -9.692467  ,  -4.9710975 ]], dtype=float32)

In [12]:
import numpy as np
from numpy import savetxt
import pandas as pd

In [13]:
arr = np.array ([x.vec for x in df2['mol2vec']])

In [14]:
savetxt('arr.csv', arr, delimiter=',')

In [15]:
arr = pd.read_csv('/content/arr.csv', sep=',', header=None) # You can also save your result as a txt file
arr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,2.388134,1.308810,-5.648465,-3.575279,-5.444581,-6.313428,-12.835257,-14.790699,-9.457028,-14.440281,15.622105,-7.795105,9.759593,-7.270654,-16.551731,-5.301712,-1.881132,9.121715,-4.922666,-7.404758,-6.191954,7.536634,-9.505957,10.259342,4.869199,-10.519434,-9.881071,-6.292707,1.151929,0.350844,-0.573598,13.886216,20.665091,2.027433,-16.460060,-20.365145,14.559703,6.364485,-8.406457,17.619207,...,-10.331122,1.850033,10.879656,-2.375796,-4.893124,-6.973983,-4.307282,25.617981,-5.141128,13.529528,0.445851,-11.215206,16.335712,-6.318654,-11.826333,-5.720281,-1.705650,7.921330,-4.323908,-10.095230,12.044325,6.044313,5.521910,-8.856503,-24.186796,11.508398,2.017782,36.668766,0.364210,-3.854083,26.554132,1.082142,2.696511,-18.553370,12.183325,-27.423279,-0.421535,20.603825,-18.554123,-3.943667
1,-5.947441,12.177692,-2.962728,-12.759828,-14.139403,-14.194730,-13.012765,-10.124955,-10.131961,-9.199222,21.992512,-14.874022,0.951873,-7.742824,-8.779722,-6.185736,-5.974085,4.093444,-9.667842,-14.478482,-11.762716,17.691496,-20.019558,4.974255,12.286263,0.670731,-3.051912,-11.179564,14.215097,2.202846,-7.222276,8.834223,23.514538,0.186866,-13.366883,-12.013205,9.043040,10.384398,-7.761697,10.617391,...,-4.128775,-0.061292,10.984462,3.698549,-11.994957,-16.565359,-6.476200,24.675306,-1.221187,17.218775,-13.639697,-3.327144,11.303439,-2.345153,-13.885582,-4.979583,4.217288,4.345410,-4.405107,-10.850863,21.187719,4.229941,6.249633,-1.719192,-21.477209,18.959467,2.713011,22.804356,-1.357382,-4.512192,23.838079,5.686640,-12.562680,-15.791921,14.878457,-33.943176,7.595187,24.682535,-12.525246,-7.381546
2,0.237571,3.458234,3.465053,-0.727978,-6.756927,-2.340914,-3.122656,-0.850639,-3.755303,-7.923760,11.395403,-5.123069,-2.245945,-5.227607,-3.409379,0.314294,-1.722746,3.161836,-3.166159,-4.077687,-5.931326,7.903691,-5.717336,3.832661,0.403444,-1.815879,0.321477,-3.196009,2.192137,-1.518397,-4.295828,7.519890,14.463423,3.542419,-1.053168,-2.455532,8.574278,7.731248,-3.886525,11.005618,...,-0.873591,-1.618285,1.710068,0.790499,0.680290,-6.451319,-0.842574,10.807609,-1.166030,4.644056,0.179578,-4.683351,5.555200,-0.363773,-6.119258,-5.785613,1.219858,2.968400,1.693540,-9.197964,8.155938,4.351836,2.147390,-1.581299,-11.340961,8.722578,3.461099,13.631584,-2.539177,-3.240337,9.898076,4.184833,-6.033554,-5.594958,4.372646,-14.478860,3.188728,10.698219,-5.596746,0.063630
3,0.349123,2.441507,-7.576409,-0.979158,-10.785616,-5.916146,-9.196584,-12.045321,-12.550273,-20.060423,26.541351,-13.109967,-0.555519,-10.568240,-9.373918,-7.602701,-5.437857,12.559546,-8.349717,-7.783520,-4.277545,7.880494,-16.882013,13.810572,-0.872607,-8.845688,-7.678116,-6.310291,1.227401,2.081449,0.517377,15.071608,23.863766,6.455638,-10.342443,-17.044600,20.407358,13.188253,-15.764236,23.099051,...,-5.906631,-1.314863,12.674661,2.983334,6.757390,-12.572453,-5.414065,27.813063,-3.195259,21.446644,0.122340,-15.610914,11.647760,-3.796439,-15.374009,-9.696092,3.866150,9.292294,5.584120,-19.468739,22.563221,12.409111,11.297673,-6.699315,-30.684170,18.460981,3.653137,38.255356,-3.053878,-0.165961,19.731110,9.113516,-7.880154,-18.929226,12.259725,-37.196793,3.708549,25.335375,-13.290848,-2.717465
4,-0.664811,6.992033,-3.284904,-3.920215,-7.320466,-9.393357,-12.481741,-11.232768,-5.029895,-5.805263,19.078913,-11.812065,1.772156,-3.535079,-13.705896,-2.848296,-5.948853,2.805779,0.576123,-13.761270,-11.369119,13.510849,-16.243549,5.700407,4.053335,-4.700745,-4.759447,-6.384858,3.144332,0.802144,-6.872624,9.687074,15.071554,5.275625,-8.644751,-8.464828,19.579248,13.362684,-12.790419,19.142448,...,-5.160031,-3.358612,12.641297,3.982446,-3.884631,-11.638659,0.232946,25.234417,-3.828568,23.084272,1.002503,-10.283107,8.248898,-2.593136,-15.183848,-11.372586,-1.